# [KDD Cup|Humanities Track Tutorial Q-Learning](https://compete.hexagon-ml.com/tutorial/kdd-cuphumanities-track-tutorial/)

### KDD Cup|Humanities Track Tutorial Q-Learning
This Tutorial builds on the previous tutorial to demonstrate a baseline implementation of a standard Reinforcement Learning (RL) Algorithm

### State

$S \in \{1,2,3,4,5\}$

### Action
$A_S = [a_{ITN},a_{IRS}]$

where  $a_{ITN} \in [0,1]$ and $a_{IRS} \in [0,1]$

### Reward
$R_{\pi} \in (- \infty,\infty)$

![](image/rewards2.png)

In [1]:
import numpy as np
from collections import defaultdict
import random

# !pip3 install git+https://github.com/slremy/netsapi --user --upgrade
from netsapi.challenge import * 

In [5]:
def actionSpace(resolution):
    x = np.arange(0,1+resolution,resolution)
    y = 1-x
    x = x.reshape(len(x),1)
    y = y.reshape(len(y),1)
    xy = np.concatenate((x, y), axis=1)
    return xy.round(2).tolist()
actionSpace(0.1)

[[0.0, 1.0],
 [0.1, 0.9],
 [0.2, 0.8],
 [0.3, 0.7],
 [0.4, 0.6],
 [0.5, 0.5],
 [0.6, 0.4],
 [0.7, 0.3],
 [0.8, 0.2],
 [0.9, 0.1],
 [1.0, 0.0]]

### Learning a Value Function Based on ϵ-greedy action selection

This common resource was used as a reference for the implementation presented here: https://kofzor.github.io/Learning_Value_Functions/. Please refer to the blog and this Tutorial in tandem. The code below uses the first example from the blog with the Challenge Environment (as opposed to Gym).

In [6]:
env = ChallengeSeqDecEnvironment(experimentCount = 105)

Q = defaultdict(lambda : 0.) # Q-function
n = defaultdict(lambda : 1.) # number of visits

def actionSpace(resolution):
    x = np.arange(0,1+resolution,resolution)
    y = 1-x
    x = x.reshape(len(x),1)
    y = y.reshape(len(y),1)
    xy = np.concatenate((x, y), axis=1)
    return xy.round(2).tolist()


#HyperParameters
epsilon = 0.1
gamma = 0.9
action_resolution = 0.1
episode_number = 1 #for submission this is fixed as 20


#Set-up
actions = actionSpace(action_resolution)
print(actions)
actionspace = range(len(actions)-1)
greedy_action = lambda s : max(actionspace, key=lambda a : Q[(s,a)])
max_q = lambda sp : max([Q[(sp,a)] for a in actionspace])

#Training of Q Table
for _ in range(episode_number):
    env.reset()
    nextstate = env.state
    while True:
        state = nextstate
        # Epsilon-Greedy
        if epsilon > random.random() :
            action = random.choice(actionspace)
            print('random_action',action)
        else :
            action = greedy_action(state)
        env_action = actions[action]#convert to ITN/IRS
        print('env_action', env_action)
        print('env_action', type(env_action))
        nextstate, reward, done, _ = env.evaluateAction(env_action)
        print('reward=',reward)
        print('done=',done)
        

        # Q-learning
        if done :
            Q[(state,action)] = Q[(state,action)] + \
                                1./n[(state,action)] * ( reward - Q[(state,action)] )
            break
        else :
            Q[(state,action)] = Q[(state,action)] + \
                                1./n[(state,action)] * ( reward + \
                                                        gamma * max_q(nextstate) - Q[(state,action)] )

#Greedy Policy Learnt from Q Table
best_policy = {state: list(actions[greedy_action(state-1)]) for state in range(1,6)}
best_reward = env.evaluatePolicy(best_policy)
print(best_policy, best_reward)

[[0.0, 1.0], [0.1, 0.9], [0.2, 0.8], [0.3, 0.7], [0.4, 0.6], [0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.8, 0.2], [0.9, 0.1], [1.0, 0.0]]
env_action [0.0, 1.0]
env_action <class 'list'>
105  Evaluations Remaining
reward= 93.54086115989467
done= False
env_action [0.0, 1.0]
env_action <class 'list'>
104  Evaluations Remaining
reward= 0.252039612048792
done= False
env_action [0.0, 1.0]
env_action <class 'list'>
103  Evaluations Remaining
reward= -0.09665503802477193
done= False
env_action [0.0, 1.0]
env_action <class 'list'>
102  Evaluations Remaining
reward= -0.17210849514804005
done= False
env_action [0.0, 1.0]
env_action <class 'list'>
101  Evaluations Remaining
reward= -0.17906876746628653
done= True
100  Evaluations Remaining
{1: [0.0, 1.0], 2: [0.0, 1.0], 3: [0.0, 1.0], 4: [0.1, 0.9], 5: [0.1, 0.9]} 115.62394762460345


### Creating a Valid Submission from Agent Code:

In [7]:
class Q_Agent():
    
    def __init__(self, environment):
        
        #Hyperparameters
        self.env = environment
        self.epsilon = 0.1
        self.gamma = 0.9
        self.action_resolution = 0.2
        self.Q = defaultdict(lambda : 0.) # Q-function
        self.n = defaultdict(lambda : 1.) # number of visits
        self.actions = actionSpace(self.action_resolution)
        self.actionspace = range(len(self.actions)-1)
        
    
    def actionSpace(self,resolution):
        x = np.arange(0,1+resolution,resolution)
        y = 1-x
        x = x.reshape(len(x),1)
        y = y.reshape(len(y),1)
        xy = np.concatenate((x, y), axis=1)
        return xy.round(2).tolist()

    def train(self):
        
        Q = self.Q
        n = self.n
        actions = self.actions
        actionspace = self.actionspace

        greedy_action = lambda s : max(actionspace, key=lambda a : Q[(s,a)])
        max_q = lambda sp : max([Q[(sp,a)] for a in actionspace])

        
        for _ in range(20): #Do not change
            
            self.env.reset()
            nextstate = self.env.state
            
            while True:
                state = nextstate

                # Epsilon-Greedy Action Selection
                if epsilon > random.random() :
                    action = random.choice(actionspace)
                else :
                    action = greedy_action(state)

                env_action = actions[action]#convert to ITN/IRS
                print('env_action', env_action)
                nextstate, reward, done, _ = self.env.evaluateAction(env_action)
                print("nextstate=",nextstate)
                print("reward=",reward)

                # Q-learning
                if done :
                    Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward - Q[(state,action)] )
                    break
                else :
                    Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward + gamma * max_q(nextstate) - Q[(state,action)] )

        return Q


    def generate(self):
        best_policy = None
        best_reward = -float('Inf')
        
        Q_trained = self.train()
        greedy_eval = lambda s : max(actionspace, key=lambda a : Q_trained[(s,a)])
        
        best_policy = {state: list(actions[greedy_eval(state-1)]) for state in range(1,6)}
        best_reward = self.env.evaluatePolicy(best_policy)
        
        print(best_policy, best_reward)
        I 
        return best_policy, best_reward

### Run the EvaluateChallengeSubmission Method with your Agent Class

In [9]:
EvaluateChallengeSubmission(ChallengeSeqDecEnvironment, Q_Agent, "Q_submission.csv")

env_action [0.0, 1.0]
105  Evaluations Remaining
nextstate= 2
reward= 108.94641992310912
env_action [0.2, 0.8]
104  Evaluations Remaining
nextstate= 3
reward= 0.06355606938390368
env_action [0.0, 1.0]
103  Evaluations Remaining
nextstate= 4
reward= 38.91473211199865
env_action [0.0, 1.0]
102  Evaluations Remaining
nextstate= 5
reward= -0.016886570501970954
env_action [0.2, 0.8]
101  Evaluations Remaining
nextstate= 6
reward= -0.011054670432794733
env_action [0.0, 1.0]
100  Evaluations Remaining
nextstate= 2
reward= 105.33250965236144
env_action [0.2, 0.8]
99  Evaluations Remaining
nextstate= 3
reward= 0.09846357813729334
env_action [0.0, 1.0]
98  Evaluations Remaining
nextstate= 4
reward= 35.49995695541685
env_action [0.2, 0.8]
97  Evaluations Remaining
nextstate= 5
reward= 0.2279525809503964
env_action [0.0, 1.0]
96  Evaluations Remaining
nextstate= 6
reward= 36.49180972972042
env_action [0.0, 1.0]
95  Evaluations Remaining
nextstate= 2
reward= 110.65601922032198
env_action [0.2, 0.8]

nextstate= 5
reward= -29.198947562882612
env_action [0.8, 0.2]
11  Evaluations Remaining
nextstate= 6
reward= -86.23689387031345
env_action [0.0, 1.0]
10  Evaluations Remaining
nextstate= 2
reward= 91.65472929701174
env_action [0.8, 0.2]
9  Evaluations Remaining
nextstate= 3
reward= 64.83425069532544
env_action [0.0, 1.0]
8  Evaluations Remaining
nextstate= 4
reward= 109.45544979599285
env_action [0.6, 0.4]
7  Evaluations Remaining
nextstate= 5
reward= 38.046884967583495
env_action [0.0, 1.0]
6  Evaluations Remaining
nextstate= 6
reward= 33.578959297425705
5  Evaluations Remaining
{1: [0.0, 1.0], 2: [0.0, 1.0], 3: [0.4, 0.6], 4: [0.0, 1.0], 5: [0.3, 0.7]} 134.37169372259612
env_action [0.0, 1.0]
105  Evaluations Remaining
nextstate= 2
reward= 108.58091464104668
env_action [0.0, 1.0]
104  Evaluations Remaining
nextstate= 3
reward= -0.042229894121489586
env_action [0.0, 1.0]
103  Evaluations Remaining
nextstate= 4
reward= 0.11935830009280002
env_action [0.0, 1.0]
102  Evaluations Remaini

nextstate= 2
reward= 95.5069022945814
env_action [0.2, 0.8]
19  Evaluations Remaining
nextstate= 3
reward= 0.2959618959254904
env_action [0.0, 1.0]
18  Evaluations Remaining
nextstate= 4
reward= 41.80830535963924
env_action [0.8, 0.2]
17  Evaluations Remaining
nextstate= 5
reward= 61.12728292093501
env_action [0.6, 0.4]
16  Evaluations Remaining
nextstate= 6
reward= 2.389299845958225
env_action [0.0, 1.0]
15  Evaluations Remaining
nextstate= 2
reward= 103.58755774366865
env_action [0.2, 0.8]
14  Evaluations Remaining
nextstate= 3
reward= 0.2014097249886988
env_action [0.0, 1.0]
13  Evaluations Remaining
nextstate= 4
reward= 35.962770406469915
env_action [0.8, 0.2]
12  Evaluations Remaining
nextstate= 5
reward= 64.40398116645541
env_action [0.6, 0.4]
11  Evaluations Remaining
nextstate= 6
reward= 1.754580359846487
env_action [0.0, 1.0]
10  Evaluations Remaining
nextstate= 2
reward= 107.37648837761975
env_action [0.2, 0.8]
9  Evaluations Remaining
nextstate= 3
reward= -0.0003050802473634

KeyboardInterrupt: 